In [1]:
from google.colab import files
files.upload_file('datasets.zip')

Saved datasets.zip to /content/datasets.zip


In [2]:
!unzip '/content/datasets.zip'

Archive:  /content/datasets.zip
   creating: datasets/
  inflating: datasets/alonso_m_full_texts.pkl  
  inflating: datasets/anderson_imbert_full_texts.pkl  
  inflating: datasets/arenal_full_texts.pkl  
  inflating: datasets/arguedas_full_texts.pkl  
  inflating: datasets/arlt_full_texts.pkl  
  inflating: datasets/arredondo_full_texts.pkl  
  inflating: datasets/arreola_full_texts.pkl  
  inflating: datasets/baldomero_full_texts.pkl  
  inflating: datasets/bareiro_saguier_full_texts.pkl  
  inflating: datasets/benedetti_full_texts.pkl  
  inflating: datasets/bioy_casares_full_texts.pkl  
  inflating: datasets/bombal_full_texts.pkl  
  inflating: datasets/borges_full_texts.pkl  
  inflating: datasets/bosch_full_texts.pkl  
  inflating: datasets/carpentier_full_texts.pkl  
  inflating: datasets/carrington_full_texts.pkl  
  inflating: datasets/cortazar_full_texts.pkl  
  inflating: datasets/dabove_full_texts.pkl  
  inflating: datasets/davila_full_texts.pkl  
  inflating: datasets/de_l

In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 64.1 MB/s eta 0:00:00


In [4]:
import os
from tqdm import tqdm
import pandas as pd
from pandas import json_normalize
import pickle

In [5]:
dirs = os.listdir('/content/datasets')

In [6]:
b = pickle.load(open('/content/datasets/mutis_full_texts.pkl', 'rb'))
b.text[0]

'Soy capitán del 3º de Lanceros de la Guardia Imperial, al mando del coronel Tadeuz Lonczynski. Voy a morir a consecuencia de las heridas que recibí en una emboscada de los desertores del Cuerpo de Zapadores de Hesse. Chapoteo en mi propia sangre cada vez que trato de volverme buscando el imposible alivio al dolor de mis huesos destrozados por la metralla. Antes de que el vidrio azul de la agonía invada mis arterias y confunda mis palabras, quiero confesar aquí mi amor, mi desordenado, secreto, inmenso, delicioso, ebrio amor por la condesa Krystina Krasinska, mi hermana. Que Dios me perdone las arduas vigilias de fiebre y deseo que pasé por ella, durante nuestro último verano en la casa de campo de nuestros padres en Katowicze. En todo instante he sabido guardar silencio. Ojalá se me tenga en cuenta en breve, cuando comparezca ante la Presencia Ineluctable. ¡Y pensar que ella rezará por mi alma al lado de su esposo y de sus hijos! FIN'

In [7]:
data = dict()
for pkl in tqdm(dirs):
  try:
      book = pickle.load(open('/content/datasets/'+pkl, 'rb'))
      for idx in range(len(book.text_metadata)):
        data[idx] = {
            'title': book.text_metadata[idx]['title'],
            'text': book.text[idx]
        }
  except:
    pass

100%|██████████| 58/58 [00:00<00:00, 1194.81it/s]


In [8]:
print(data[0])

{'title': 'El acto libre', 'text': 'La Secretaria privada del Señor X, Director General de la Confederación Internacional de la Producción Universal, entró tímidamente en su privadísimo despacho con una tarjeta en la mano. Se la entregó balbuceando. -Es un señor que solicita hablar con Ud. -¡Que vuelva otro día! ¡Estoy ocupado! -Es que este señor ha estado viniendo, día a día, desde hace ocho meses, don Alcibíades. -¡Ah! ¿Sí? ¡Y cómo no me lo había dicho antes! ¿De qué se trata? -No quiere decir. Asegura que es un asunto privado. X pensó un poco; luego, botando la tarjeta al canasto sin mirarla, decidió: -Hágalo entrar. La verdad era que, en ese momento, no tenía nada que hacer. Casi al instante apareció un viejito semijorobado, con un inmenso cartapacio debajo del brazo. Hizo una reverencia y se sentó frente al inaccesible magnate. -¿En qué puedo servirle? -rugió X con una voz que estaba a punto de dar por terminada la entrevista. -En nada. -¿Cómo en nada? -Soy yo el que puede servirl

In [9]:
cuentos = pd.DataFrame.from_dict(data, orient='index')

In [10]:
cuentos.shape

(62, 2)

In [11]:
textos = cuentos[['text']]

In [12]:
textos.head()

,text
0,"La Secretaria privada del Señor X, Director Ge..."
1,Un accidente sin importancia (la transitoria d...
2,"El asunto ocurrió hace mucho tiempo, cuando yo..."
3,"–Usted fue, sí, usted fue. –¿Señora…? –Le digo..."
4,"“Anoche, a las doce y media próximamente, el c..."


In [13]:
from sklearn.model_selection import train_test_split # VALIDACION Y TEST SPLITEADOS EN UN PORCENTAJE
import re # LIBRERIA PARA TRABAJAR CON EXPRESIONES REGULARES

In [14]:
# CON ESTA FUNCIONALIDAD PRETENDEMOS CREAR UN ARCHIVO DE TRAIN Y TEST CON VECTORES QUE INDIQUEN LOS TEXTOS TOKENIZADOS
def build_text_files(cuentos, destino):
    f = open(destino, 'w')
    data = ''
    for cuento in cuentos:
        try:
          summary = str(cuento).strip()
          summary = re.sub(r"\s", " ", summary)
          data += summary + "  "
        except:
          pass
    f.write(data)
    return data


In [15]:
train, test = train_test_split(cuentos,test_size=0.20) # TOMAMOS EL 15% DE NUESTRA DATA PARA TEST

In [16]:
build_text_files(train['text'].values,'/content/train_dataset.txt')
build_text_files(test['text'].values,'/content/test_dataset.txt')

print("Train dataset length: "+str(len(train))) # VISUALIZAMOS LA LONGITUD DE NUESTRO DATASET
print("Test dataset length: "+ str(len(test)))

Train dataset length: 49
Test dataset length: 13


In [17]:
from transformers import AutoTokenizer # IMPORTAMOS NUESTRO TOKENIZER BASANDONOS EN UN MODELO ESPANYOL EN GPT2

tokenizer = AutoTokenizer.from_pretrained("DeepESP/gpt2-spanish")

train_path = '/content/train_dataset.txt'
test_path = '/content/test_dataset.txt'

In [24]:
from transformers import TextDataset,DataCollatorForLanguageModeling # IMPORTAMOS NUESTRA UTILERIA PARA TRABAJAR CON TEXTOS Y PASARLOS A VECTORES

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
        overwrite_cache=True,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
        overwrite_cache=True,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

In [26]:
train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer) # LLAMAMOS A NUESTRA FUNCION

In [29]:
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead # IMPORTAMOS EL ENTRENADOR, EL BUILD DE ARGUMENTOS Y EL MODELO CON MULTIPLE LAYERS HEADS

model = AutoModelWithLMHead.from_pretrained("DeepESP/gpt2-spanish")
# CREAMOS NUESTROS ARGUMENTOS
training_args = TrainingArguments(
    output_dir="./gpt2-cuentos", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=15, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )

# CREAMOS NUESTRO ENTRENADOR
trainer = Trainer(
    model=model, #MODELO
    args=training_args, # ARGUMENTOS
    data_collator=data_collator, #ALLOCATOR
    train_dataset=train_dataset, #TRAIN
    eval_dataset=test_dataset, #TEST
    # prediction_loss_only=True,

)

/usr/local/lib/python3.9/dist-packages/transformers/models/auto/modeling_auto.py:1295: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [30]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=390, training_loss=3.604100310496795, metrics={'train_runtime': 410.3566, 'train_samples_per_second': 29.645, 'train_steps_per_second': 0.95, 'total_flos': 794654392320000.0, 'train_loss': 3.604100310496795, 'epoch': 15.0})

In [31]:
trainer.save_model()

In [34]:
from transformers import pipeline

In [35]:
storyTeller = pipeline("text-generation", model="./gpt2-cuentos", tokenizer="DeepESP/gpt2-spanish")

In [37]:
storyteller = storyTeller("Erase una vez")[0]["generated_text"]
print(storyteller)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Erase una vez a uno de los bandidos que se sentaba al pie de un pequeño estrado que había allí al lado del estrado con ese aire de indiferencia impropio de un individuo que lo ignora particularmente.” * * El hombre pareció desconcertado porque la
